The following is from [this article](https://medium.com/daily-programming-tips/getting-started-with-python-and-influxdb-3e24e3f6b585) in Medium and [Getting Started Instruction](https://docs.influxdata.com/influxdb/v2/install/) for InfluxDB.

In [1]:
import os
import time

import influxdb_client
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

token = os.environ.get("INFLUXDB_TOKEN")
org = "test"
url = "http://localhost:8086"

client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)

In [2]:
bucket = "test_db"

write_api = client.write_api(write_options=SYNCHRONOUS)

for value in range(5):
    point = Point("measurement1").tag("tagname1", "tagvalue1").field("field1", value)
    write_api.write(bucket=bucket, org="test", record=point)
    time.sleep(1)  # separate points by 1 second

In [3]:
query_api = client.query_api()

query = """from(bucket: "test_db")
 |> range(start: -10m)
 |> filter(fn: (r) => r._measurement == "measurement1")"""
tables = query_api.query(query, org="test")

for table in tables:
    for record in table.records:
        print(record)

FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2024, 3, 16, 12, 7, 26, 33317, tzinfo=tzutc()), '_stop': datetime.datetime(2024, 3, 16, 12, 17, 26, 33317, tzinfo=tzutc()), '_time': datetime.datetime(2024, 3, 16, 12, 16, 1, 86164, tzinfo=tzutc()), '_value': 0, '_field': 'field1', '_measurement': 'measurement1', 'tagname1': 'tagvalue1'}
FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2024, 3, 16, 12, 7, 26, 33317, tzinfo=tzutc()), '_stop': datetime.datetime(2024, 3, 16, 12, 17, 26, 33317, tzinfo=tzutc()), '_time': datetime.datetime(2024, 3, 16, 12, 16, 2, 88263, tzinfo=tzutc()), '_value': 1, '_field': 'field1', '_measurement': 'measurement1', 'tagname1': 'tagvalue1'}
FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2024, 3, 16, 12, 7, 26, 33317, tzinfo=tzutc()), '_stop': datetime.datetime(2024, 3, 16, 12, 17, 26, 33317, tzinfo=tzutc()), '_time': datetime.datetime(2024, 3, 16, 12,

In [4]:
query_api = client.query_api()

query = """from(bucket: "test_db")
  |> range(start: -10m)
  |> filter(fn: (r) => r._measurement == "measurement1")
  |> mean()"""
tables = query_api.query(query, org="test")

for table in tables:
    for record in table.records:
        print(record)

FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2024, 3, 16, 12, 7, 26, 52262, tzinfo=tzutc()), '_stop': datetime.datetime(2024, 3, 16, 12, 17, 26, 52262, tzinfo=tzutc()), '_value': 2.0, '_field': 'field1', '_measurement': 'measurement1', 'tagname1': 'tagvalue1'}
